In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np

In [2]:
url = 'http://www.boxofficemojo.com/movies/?page=intl&id=easya.htm&sort=percoftotal&order=DESC&p=.htm'
def scrape_page(url):
    response = requests.get(url)
    try:
        soup = BeautifulSoup(response.text)
        return soup
    except:
        return None

In [3]:
easya = scrape_page(url)

In [29]:
movie_title = ""
movie_data = []

In [30]:
movie_title = easya.find('title').text.split(" (")[0]

In [6]:
header = easya.find_all('td')[4]
international = easya.find_all('table')[7]

In [7]:
# header

In [8]:
def get_header_value(table, field_name):
    '''Grab a value from boxofficemojo HTML
    
    Takes a string attribute of a movie on the page and
    returns the string in the next sibling object
    (the value for that attribute)
    or None if nothing is found.
    '''
    obj = table.find(text=re.compile(field_name))
    # this works for most of the values
    next_sibling = obj.findNextSibling()
    if next_sibling:
        return next_sibling.text 
    else:
        return ''

In [31]:
dtg = get_header_value(header,'Domestic Total')
dist = get_header_value(header,'Distributor')
release = get_header_value(header, 'Release Date')
genre = get_header_value(header, 'Genre')
runtime = get_header_value(header,'Runtime')
budget = get_header_value(header, 'Budget')
rating = get_header_value(header,'MPAA Rating')

movie_data.append(movie_title)
movie_data.append(dtg)
movie_data.append(dist)
movie_data.append(release)
movie_data.append(genre)
movie_data.append(runtime)
movie_data.append(budget)
movie_data.append(rating)

In [10]:
# movie_data

In [19]:
countries = international.find_all('tr')
foreign_total = []
hungary = []

In [14]:
# do this for foreign_total and for Hungary
# e.g. get_international('Hungary', hungary)
def get_international(country, country_list):
    country_data = ''
    for row in countries:
        data = row.find_all('td')
        if data[0].find(text=country):
            country_data = row
    #get country name, insert as first element to list
    country_list.append(country_data.find('a').contents[0])
    columns = country_data.find_all('td')[1:]
    for column in columns:
        country_list.append(column.get_text())
    return country_list

In [21]:
foreign_total = get_international('FOREIGN TOTAL', foreign_total)
hungary = get_international('Hungary', hungary)

In [74]:
df = pd.DataFrame(columns=('title', 'dtg', 'dist', 'release', 'genre', 'runtime', 'budget', 'rating', 'foreign', 'foreign_dist', 'foreign_release', 'foreign_opening', 'foreign_pct', 'foreign_total', 'foreign_asof', 'hungary', 'hun_dist', 'hun_release', 'hun_opening', 'hun_pct', 'hun_total', 'hun_asof'))

In [76]:
row = movie_data+foreign_total+hungary

In [77]:
row_s = pd.Series(row, index=df.columns)

In [79]:
df = df.append(row_s, ignore_index=True)

In [80]:
df

,title,dtg,dist,release,genre,runtime,budget,rating,foreign,foreign_dist,...,foreign_pct,foreign_total,foreign_asof,hungary,hun_dist,hun_release,hun_opening,hun_pct,hun_total,hun_asof
0,Easy A,"$58,401,464",Sony / Screen Gems,"September 17, 2010",Romantic Comedy,1 hrs. 32 min.,$8 million,PG-13,FOREIGN TOTAL,-,...,6.8%,"$16,550,841",3/20/11,Hungary,InterCom,10/28/10,"$47,154",40.6%,"$116,150",11/7/10
